In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [2]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 8MjNrrs9WB53nQfmLhWeg1G6iAC6NxSjTVQUZZt3F77G, 1>


# Louvain algorithm

In [5]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [7]:

#  We use this counter to generate a unique column name below 
#
l_cntr += 1
   #
l_output_propname     = "louvain_" + str(l_cntr).zfill(4)


l_result = analytics.louvain_clustering(
   my_graph,
      #
   result_property_name = l_output_propname,
      #
   is_symmetric         = True
   )

print(l_result)
   #
print("--")


          0/? [?op/s]

None
--


In [ ]:

---------------------------------------------------------------------------
CanceledError                             Traceback (most recent call last)
Cell In[12], line 8
      4    #
      5 l_output_propname     = "louvain_" + str(l_cntr).zfill(4)
----> 8 l_result = analytics.louvain_clustering(
      9    my_graph,
     10       #
     11    result_property_name = l_output_propname,
     12       #
     13    is_symmetric         = True
     14    )
     16 print(l_result)
     17    #

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:168, in async_to_sync.<locals>.wrapper(timeout, *args, **kwargs)
    165 if convert_objects:
    166     registry = AsyncToSyncClassRegistry.get()
    167     return registry.async_to_sync(
--> 168         wait_for(
    169             async_func(
    170                 *(registry.sync_to_async(a) for a in args),
    171                 **{k: registry.sync_to_async(v) for k, v in kwargs.items()},
    172             ),
    173             timeout=timeout,
    174         )
    175     )
    176 return wait_for(async_func(*args, **kwargs), timeout=timeout)

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:147, in wait_for(coro, timeout)
    145 try:
    146     future = asyncio.run_coroutine_threadsafe(timeout_coro, loop=AsyncRunnerThread.get().loop)
--> 147     return future.result()
    148 except KeyboardInterrupt:
    149     inner_future.cancel()

File /opt/conda/lib/python3.8/concurrent/futures/_base.py:444, in Future.result(self, timeout)
    442     raise CancelledError()
    443 elif self._state == FINISHED:
--> 444     return self.__get_result()
    445 else:
    446     raise TimeoutError()

File /opt/conda/lib/python3.8/concurrent/futures/_base.py:389, in Future.__get_result(self)
    387 if self._exception:
    388     try:
--> 389         raise self._exception
    390     finally:
    391         # Break a reference cycle with the exception in self._exception
    392         self = None

File /opt/conda/lib/python3.8/asyncio/tasks.py:455, in wait_for(fut, timeout, loop)
    450     warnings.warn("The loop argument is deprecated since Python 3.8, "
    451                   "and scheduled for removal in Python 3.10.",
    452                   DeprecationWarning, stacklevel=2)
    454 if timeout is None:
--> 455     return await fut
    457 if timeout <= 0:
    458     fut = ensure_future(fut, loop=loop)

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/analytics/__init__.py:256, in louvain_clustering(graph, result_property_name, edge_weight_property_name, is_symmetric)
    234 """
    235 Run Louvain Clustering on a symmetric ``graph`` and write the cluster to the property named by
    236 ``result_property_name``. The graph must have at least 100 nodes.
   (...)
    244        to false by default.
    245 """
    246 data = {
    247     "analytics": {
    248         "louvain_clustering": {
   (...)
    253     },
    254 }
--> 256 await graph._run_asynchronously(data)

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/graph.py:322, in Graph._run_asynchronously(self, data)
    319 async def _run_asynchronously(self, data):
    320     data = self._add_operation_metadata(data)
--> 322     result = (await self._client._run_on_graph_and_wait(self, data))["result"]
    323     await self._client._refresh_graph_version(self)
    324     return result

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:333, in Database._run_on_graph_and_wait(self, graph, data)
    331 while attempt < max_attempts:
    332     try:
--> 333         return await self._wait_op(op)
    334     except errors.ServerCommunicationError:
    335         logging.warning(f"error while waiting, retrying (attempt={attempt})")

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/client.py:270, in Database._wait_op(self, op)
    268 async with _progress_bar() as pbar:
    269     async with wait_fn(operation_id) as stream:
--> 270         async for update in stream:
    271             status = update["status"]
    273             for entry in update.get("progress", []):

File /opt/conda/lib/python3.8/site-packages/asyncstdlib/builtins.py:445, in map(function, *iterable)
    443 async for args in args_iter:
    444     result = function(*args)
--> 445     yield await result

File /opt/conda/lib/python3.8/site-packages/asyncstdlib/_core.py:134, in force_async.<locals>.async_wrapped(*args, **kwargs)
    133 async def async_wrapped(*args: Any, **kwargs: Any) -> T:
--> 134     return call(*args, **kwargs)

File /opt/conda/lib/python3.8/site-packages/katana_enterprise/rpc/client.py:171, in _OperationClient._event_stream.<locals>.parse_stream(line)
    169     raise errors.OperationError(operation_id + " " + data["status_message"])
    170 if status == "Canceled":
--> 171     raise errors.CanceledError()
    173 return data

CanceledError: 
        

# Checking that result above


In [8]:

#  Unfiltered node data
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n
   ORDER BY n.{0} DESC
    
    """.format(l_output_propname)

l_result = my_graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data, after adding Narita and Seoul below-
#
#     +----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------+
#     |    |       n.louvain_0023 |           ID(n) |   n.internal_id | n.labels    | n.airport_code   | n.airport_name   |       n.louvain_0023 | n.type   | n.LABEL   |
#     |----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------|
#     |  0 | 18446744073709551615 |               2 |               2 | ['Airport'] | NRT              | Narita, Japan    | 18446744073709551615 | node     | nan       |
#     |  1 | 18446744073709551615 |               0 |               0 | ['Airport'] | ORD              | Chicago O-Hare   | 18446744073709551615 | node     | Airport   |
#     |  2 | 18446744073709551615 |               1 |               1 | ['Airport'] | SJC              | San Jose         | 18446744073709551615 | node     | Airport   |
#     |  3 | 18446744073709551615 | 281474976710656 | 281474976710656 | ['Airport'] | ICN              | Seoul, Korea     | 18446744073709551615 | node     | nan       |
#     |  4 | 18446744073709551615 | 562949953421312 | 562949953421312 | ['Airport'] | MKE              | Milwaukee        | 18446744073709551615 | node     | Airport   |
#     |  5 | 18446744073709551615 | 562949953421313 | 562949953421313 | ['Airport'] | DEN              | Denver           | 18446744073709551615 | node     | Airport   |
#     +----+----------------------+-----------------+-----------------+-------------+------------------+------------------+----------------------+----------+-----------+


          0/? [?op/s]

+----+----------------------+-----------------+-----------------+-------------+-----------+------------------+------------------+----------------------+----------+
|    |       n.louvain_0022 |           ID(n) |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   |       n.louvain_0022 | n.type   |
|----+----------------------+-----------------+-----------------+-------------+-----------+------------------+------------------+----------------------+----------|
|  0 | 18446744073709551615 |               0 |               0 | ['Airport'] | Airport   | MKE              | Milwaukee        | 18446744073709551615 | node     |
|  1 | 18446744073709551615 |               1 |               1 | ['Airport'] | Airport   | DEN              | Denver           | 18446744073709551615 | node     |
|  2 | 18446744073709551615 | 281474976710656 | 281474976710656 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | 18446744073709551615 | node     |
|  3 | 184467440

/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


In [ ]:

#  View output
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n.airport_name
   ORDER BY n.{0} DESC
   LIMIT 10
    
    """.format(l_output_propname)

l_result = my_graph.query(l_query)
   #    
print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

#  Add a new community
#

l_query = """
   CREATE ( n: Airport { airport_code: 'NRT' } )               //  Notice single curly braces
   SET n.airport_name = 'Narita, Japan' 
   """
      #
display(l_result = my_graph.query(l_query))

l_query = """
   CREATE ( n: Airport { airport_code: 'ICN' } ) 
   SET n.airport_name = 'Seoul, Korea' 
   """
      #
display(l_result = my_graph.query(l_query))


l_query = """
   MATCH
      (n: Airport   ),
      (m: Airport   )
   WHERE n.airport_code     = 'NRT' AND m.airport_code  = 'ICN'
   CREATE (n) -[r: FLIES_TO { DISTANCE: 752, NUM_HOPS: 1}]-> (m)
   """
      #
display(l_result = my_graph.query(l_query))



In [ ]:
#  Go up, run Louvain again ..